# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 17 2022 8:30AM,244312,10,0085939288,ISDIN Corporation,Released
1,Aug 17 2022 8:30AM,244312,10,0085939289,ISDIN Corporation,Released
2,Aug 17 2022 8:30AM,244312,10,0085939290,ISDIN Corporation,Released
3,Aug 17 2022 8:30AM,244312,10,0085939293,ISDIN Corporation,Released
4,Aug 17 2022 8:30AM,244312,10,0085939292,ISDIN Corporation,Released
5,Aug 17 2022 8:30AM,244312,10,0085939291,ISDIN Corporation,Released
6,Aug 17 2022 8:30AM,244312,10,0085939306,ISDIN Corporation,Released
7,Aug 17 2022 8:30AM,244312,10,0085939307,ISDIN Corporation,Released
8,Aug 17 2022 8:30AM,244312,10,0085939308,ISDIN Corporation,Released
9,Aug 17 2022 8:30AM,244312,10,0085939309,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,244319,Released,1
35,244320,Released,1
36,244321,Released,1
37,244322,Released,1
38,244323,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244319,NaN,NaN,1.0
244320,NaN,NaN,1.0
244321,NaN,NaN,1.0
244322,NaN,NaN,1.0
244323,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244215,65.0,4.0,5.0
244216,28.0,0.0,10.0
244222,9.0,0.0,3.0
244245,5.0,3.0,13.0
244251,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244215,65,4,5
244216,28,0,10
244222,9,0,3
244245,5,3,13
244251,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244215,65,4,5
1,244216,28,0,10
2,244222,9,0,3
3,244245,5,3,13
4,244251,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244215,65,4,5
1,244216,28,,10
2,244222,9,,3
3,244245,5,3,13
4,244251,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 17 2022 8:30AM,244312,10,ISDIN Corporation
21,Aug 17 2022 8:30AM,244313,10,"Nextsource Biotechnology, LLC"
23,Aug 17 2022 8:30AM,244316,10,TherapeuticsMD Retain
24,Aug 17 2022 8:30AM,244318,10,Therapeutics Retain 2
25,Aug 17 2022 8:30AM,244319,10,TherapeuticsMD Sample
26,Aug 17 2022 8:30AM,244322,10,"TherapeuticsMD, Inc."
27,Aug 16 2022 5:41PM,244323,21,"NBTY Global, Inc."
28,Aug 16 2022 5:27PM,244321,21,"NBTY Global, Inc."
29,Aug 16 2022 5:26PM,244320,102,TherapeuticsMD Sample
30,Aug 16 2022 5:15PM,244317,17,Proven Skincare


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 17 2022 8:30AM,244312,10,ISDIN Corporation,,,21
1,Aug 17 2022 8:30AM,244313,10,"Nextsource Biotechnology, LLC",,,2
2,Aug 17 2022 8:30AM,244316,10,TherapeuticsMD Retain,,,1
3,Aug 17 2022 8:30AM,244318,10,Therapeutics Retain 2,,,1
4,Aug 17 2022 8:30AM,244319,10,TherapeuticsMD Sample,,,1
5,Aug 17 2022 8:30AM,244322,10,"TherapeuticsMD, Inc.",,,1
6,Aug 16 2022 5:41PM,244323,21,"NBTY Global, Inc.",,,1
7,Aug 16 2022 5:27PM,244321,21,"NBTY Global, Inc.",,,1
8,Aug 16 2022 5:26PM,244320,102,TherapeuticsMD Sample,,,1
9,Aug 16 2022 5:15PM,244317,17,Proven Skincare,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 8:30AM,244312,10,ISDIN Corporation,21,,
1,Aug 17 2022 8:30AM,244313,10,"Nextsource Biotechnology, LLC",2,,
2,Aug 17 2022 8:30AM,244316,10,TherapeuticsMD Retain,1,,
3,Aug 17 2022 8:30AM,244318,10,Therapeutics Retain 2,1,,
4,Aug 17 2022 8:30AM,244319,10,TherapeuticsMD Sample,1,,
5,Aug 17 2022 8:30AM,244322,10,"TherapeuticsMD, Inc.",1,,
6,Aug 16 2022 5:41PM,244323,21,"NBTY Global, Inc.",1,,
7,Aug 16 2022 5:27PM,244321,21,"NBTY Global, Inc.",1,,
8,Aug 16 2022 5:26PM,244320,102,TherapeuticsMD Sample,1,,
9,Aug 16 2022 5:15PM,244317,17,Proven Skincare,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 8:30AM,244312,10,ISDIN Corporation,21,,
1,Aug 17 2022 8:30AM,244313,10,"Nextsource Biotechnology, LLC",2,,
2,Aug 17 2022 8:30AM,244316,10,TherapeuticsMD Retain,1,,
3,Aug 17 2022 8:30AM,244318,10,Therapeutics Retain 2,1,,
4,Aug 17 2022 8:30AM,244319,10,TherapeuticsMD Sample,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 8:30AM,244312,10,ISDIN Corporation,21.0,NaN,NaN
1,Aug 17 2022 8:30AM,244313,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
2,Aug 17 2022 8:30AM,244316,10,TherapeuticsMD Retain,1.0,NaN,NaN
3,Aug 17 2022 8:30AM,244318,10,Therapeutics Retain 2,1.0,NaN,NaN
4,Aug 17 2022 8:30AM,244319,10,TherapeuticsMD Sample,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 8:30AM,244312,10,ISDIN Corporation,21.0,0.0,0.0
1,Aug 17 2022 8:30AM,244313,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
2,Aug 17 2022 8:30AM,244316,10,TherapeuticsMD Retain,1.0,0.0,0.0
3,Aug 17 2022 8:30AM,244318,10,Therapeutics Retain 2,1.0,0.0,0.0
4,Aug 17 2022 8:30AM,244319,10,TherapeuticsMD Sample,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1954503,41.0,0.0,0.0
102,244320,1.0,0.0,0.0
12,244298,1.0,0.0,1.0
15,732653,18.0,4.0,102.0
16,1221423,5.0,0.0,0.0
17,244317,1.0,0.0,0.0
19,488607,1.0,13.0,6.0
20,977028,50.0,4.0,5.0
21,732959,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1954503,41.0,0.0,0.0
1,102,244320,1.0,0.0,0.0
2,12,244298,1.0,0.0,1.0
3,15,732653,18.0,4.0,102.0
4,16,1221423,5.0,0.0,0.0
5,17,244317,1.0,0.0,0.0
6,19,488607,1.0,13.0,6.0
7,20,977028,50.0,4.0,5.0
8,21,732959,3.0,0.0,0.0
9,22,732772,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,41.0,0.0,0.0
1,102,1.0,0.0,0.0
2,12,1.0,0.0,1.0
3,15,18.0,4.0,102.0
4,16,5.0,0.0,0.0
5,17,1.0,0.0,0.0
6,19,1.0,13.0,6.0
7,20,50.0,4.0,5.0
8,21,3.0,0.0,0.0
9,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,41.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,18.0
4,16,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21,22
Status,,,,,,,,,,
Completed,0.0,0.0,1.0,102.0,0.0,0.0,6.0,5.0,0.0,0.0
Executing,0.0,0.0,0.0,4.0,0.0,0.0,13.0,4.0,0.0,0.0
Released,41.0,1.0,1.0,18.0,5.0,1.0,1.0,50.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,0.0,0.0,1.0,102.0,0.0,0.0,6.0,5.0,0.0,0.0
1,Executing,0.0,0.0,0.0,4.0,0.0,0.0,13.0,4.0,0.0,0.0
2,Released,41.0,1.0,1.0,18.0,5.0,1.0,1.0,50.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,0.0,0.0,1.0,102.0,0.0,0.0,6.0,5.0,0.0,0.0
1,Executing,0.0,0.0,0.0,4.0,0.0,0.0,13.0,4.0,0.0,0.0
2,Released,41.0,1.0,1.0,18.0,5.0,1.0,1.0,50.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()